In [2]:
import m2
import numpy as np
import pickle
from Agent import MaxAgent
import math



In [25]:

f = open('playdata/balvin1', 'rb')
data = pickle.load(f)[1:][0]
print(data)
stateActions = []
for state, action in data:
    stateActions.append((state, action))
f = open('playdata/balvin2', 'rb')
data = pickle.load(f)[1:][0]
for state, action in data:
    stateActions.append((state, action))
f = open('playdata/g1', 'rb')
data = pickle.load(f)
data = data['playthrough'][0]
states, actions = data
for i in range(len(actions)):
    stateActions.append((states[i], actions[i]))
f = open('playdata/g2', 'rb')
data = pickle.load(f)
data = data['playthrough'][0]
states, actions = data
for i in range(len(actions)):
    stateActions.append((states[i], actions[i]))
stateActions

[(Cash: 0, (1, 5, 'move')), (Cash: 0, (3, 9, 'move')), (Cash: 0, (3, 2, 'move')), (Cash: 0, (0, 0, 'move')), (Cash: 47, (45, 45, 'tank')), (Cash: 2, (19, 2, 'move')), (Cash: 2, (0, 0, 'move')), (Cash: 153, (None, None, 'exit'))]


[(Cash: 0, (1, 5, 'move')),
 (Cash: 0, (3, 9, 'move')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (0, 0, 'move')),
 (Cash: 47, (45, 45, 'tank')),
 (Cash: 2, (19, 2, 'move')),
 (Cash: 2, (0, 0, 'move')),
 (Cash: 153, (None, None, 'exit')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (4, 5, 'move')),
 (Cash: 0, (3, 7, 'move')),
 (Cash: 0, (5, 9, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 62, (11, 9, 'move')),
 (Cash: 62, (0, 9, 'move')),
 (Cash: 91, (45, 45, 'tank')),
 (Cash: 46, (9, 6, 'move')),
 (Cash: 46, (13, 4, 'move')),
 (Cash: 46, (10, 2, 'move')),
 (Cash: 46, (8, 1, 'move')),
 (Cash: 46, (0, 0, 'move')),
 (Cash: 183, (None, None, 'exit')),
 (Cash: 0, (4, 5, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 15, (0, 0, 'move')),
 (Cash: 15, (13, 4, 'move')),
 (Cash: 15, (16, 3, 'move')),
 (Cash: 15, (19, 7, 'move')),
 (Cash: 0, (8, 1, 'move')),
 (Cash: 0, (10, 2, 'move')),
 (Cash: 0, (0, 0, 'move')),
 (Cash: 78, (60, 45, 'tank')),
 (Cash: 18, (16, 3, 'move')),
 (Cash: 18, (13, 4, 'move')),


In [28]:
def generateKdata(k):
    data = []
    agent = MaxAgent(depth = k - 1)
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            reward += agent.value(nextState)[0]
            values.append(reward)
        data.append((index, values))
    return data   
                
        
        

In [29]:
def generateQdata(iters):
    data = []
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            _, _, scores, _ = m2.getRollout(nextState, iters)
            values.append(scores[-1])
        data.append((index, values))
    return data   

In [31]:
qdata = generateQdata(100000)

In [30]:
kdata = generateKdata(3)

In [310]:
k3data = kdata

In [32]:
k2data = generateKdata(2)

In [33]:
k1data = generateKdata(1)

In [34]:
def calculate(data, alpha):
    arr = []
    for a, b in data:
        total = 0
        for num in b:
            total += pow(math.e, alpha * num)
        if total == 0:
            arr.append(1 / len(b))
        else:
            arr.append(pow(math.e, alpha * b[a]) / total)
    return np.array(arr, dtype=np.float64)

In [35]:
karr = calculate(kdata, 0.1)

In [36]:
qarr = calculate(qdata, 0.04)

In [37]:
mat = np.column_stack((karr, qarr))

In [39]:
import numpy as np
from scipy.optimize import minimize

m = mat
b = np.array([1 for _ in range(39)])


def loss(x):
    return np.sum(np.square((np.dot(mat, x) - b)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})

res = minimize(loss, x0 =[0,0], method='SLSQP', constraints=cons, options={'disp': True})

print(res.x)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 18.123147079077146
            Iterations: 3
            Function evaluations: 13
            Gradient evaluations: 3
[-0.012682  1.012682]


In [40]:
def loss(x):
    return -np.sum(np.log(np.dot(mat, x)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0},
       {'type': 'ineq',
         'fun' : lambda x: x[0]},
        {'type': 'ineq',
         'fun' : lambda x: x[1]}
       )

res = minimize(loss, x0 =[0.1,0.9], method='SLSQP', constraints=cons, options={'disp': True})
print(res.x)


# argmax_model P(model | F1 F2 F3 F4 F5 ...)
#  = argmax_model P(model F1 F2 F3 ...)
#  = argmax_model P(F1 F2 F3 ... | model)P(model)
#  = argmax_model P(F1 | model) * .... P(model)
    
#  ~argmax_model P(F1 F2 F3 ... | model)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 59.974405855205305
            Iterations: 5
            Function evaluations: 21
            Gradient evaluations: 5
[0.30471432 0.69528568]


In [41]:
def mix(datas, weights):
    data = []
    for i in range(len(datas[0])):
        total = [0 for _ in range(len(datas[0][i][1]))]
        for j in range(len(datas)):
            for k in range(len(datas[j][i][1])):
                total[k] += weights[j] * datas[j][i][1][k]
        data.append((datas[0][i][0], total))
    return data
#i is sample index, j is dataset index, k is value index
    

In [44]:
def loss(x):
    return np.sum(np.square(calculate(mix([k1data, k2data, kdata, qdata], x[0:4]), x[4]) - b))
#x[0:4] are the linear weights
#x[4] is the alpha value
#mix applies the weights to the valuations of the actions by the different agents and combines them into one
#calculate does the softmax probability calculation based on x[4], the action chosen, and the valuations of each action

b = np.array([1 for _ in range(39)])

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x[0:4]) - 1.0},
       {'type': 'ineq',
         'fun' : lambda x: x[0]},
        {'type': 'ineq',
         'fun' : lambda x: x[1]},
       {'type': 'ineq',
         'fun' : lambda x: x[2]},
        {'type': 'ineq',
         'fun' : lambda x: x[3]})

res = minimize(loss, x0 =[0.25, 0.25, 0.25, 0.25, 0.1], method='SLSQP', constraints=cons, options={'disp': True})

print(res.x)


C:\Users\Sherman Luo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  
C:\Users\Sherman Luo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Sherman Luo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 16.3135709587984
            Iterations: 32
            Function evaluations: 259
            Gradient evaluations: 32
[0.16557716 0.2639918  0.33473297 0.23569807 4.14451725]
